In [ ]:
#All imports/installs needed

!pip install qiskit-algorithms
!pip install qiskit_machine_learning
!pip install qiskit_ibm_runtime
!pip install qiskit-aer
!pip install kagglehub
!pip install qiskit-aer-gpu
!pip install qiskit-aer-gpu-cu11
!pip install pylatexenc
import qiskit
import matplotlib as plt 
import matplotlib.pyplot as plt 
import numpy as np
import kagglehub
import mpl_toolkits.mplot3d  # noqa: F401
from qiskit.primitives import Sampler
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes 
from qiskit_ibm_runtime import qiskit_runtime_service, SamplerV2 as Sampler
from qiskit_machine_learning.algorithms import QSVC
from qiskit import QuantumRegister,ClassicalRegister,QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_machine_learning.algorithms import VQC as MLVQC
from qiskit_algorithms.optimizers import COBYLA
from qiskit import transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator





ERROR: Could not find a version that satisfies the requirement qiskit-aer-gpu (from versions: none)
ERROR: No matching distribution found for qiskit-aer-gpu
ERROR: Could not find a version that satisfies the requirement qiskit-aer-gpu-cu11 (from versions: none)
ERROR: No matching distribution found for qiskit-aer-gpu-cu11


In [16]:
#sklearn and matplotlib 

from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score
from sklearn.decomposition import KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [13]:
# Download dataset using kagglehub 
path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")

print("Path to dataset files:", path)
import pandas as pd
data = pd.read_csv(path + "/data.csv")
data.head()

data = data.drop(columns=[ "Unnamed: 32"])


se_columns = [col for col in data.columns if '_se' in col]

data_display = ['id', 'diagnosis'] + se_columns 

data[data_display].head()

Path to dataset files: C:\Users\lhlee\.cache\kagglehub\datasets\uciml\breast-cancer-wisconsin-data\versions\2


,id,diagnosis,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se
0,842302,M,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193
1,842517,M,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532
2,84300903,M,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571
3,84348301,M,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208
4,84358402,M,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115


In [14]:
# Extract standard error (se) features from the dataset
se_features = [col for col in data.columns if '_se' in col]

#define the features and labels 
X = data[se_features].values
y = data['diagnosis'].map({'M': 1, 'B': 0}).values   #M : Malignant, B: Benign
m_count = np.sum(y)
b_count = len(y) - m_count

#print out such features and labels 
print("Total amount of Malignant samples: " , m_count)
print("Total amount of Benign samples: ", b_count)
print("Totals Samples: ", X.shape[0])
print("Total Features: ", X.shape[1])
plt.show()


Total amount of Malignant samples:  212
Total amount of Benign samples:  357
Totals Samples:  569
Total Features:  10


In [17]:
#Using a classical method run a train test split

algorithm_globals.random_seed = 123
train_features, test_features, train_labels, test_labels = train_test_split(
    X, y, train_size=0.8, # standard 80/20 train test split
    test_size=0.2, 
    random_state=42
)

svc = SVC(random_state=42, kernel='rbf', C=1.0)
_= svc.fit(train_features, train_labels)

train_score_c4 = svc.score(train_features, train_labels)
test_score_c4 = svc.score(test_features, test_labels)

print(f"Classical SVC on the training dataset: {train_score_c4:.2f}")
print(f"Classical SVC on the test dataset:     {test_score_c4:.2f}")

Classical SVC on the training dataset: 0.85
Classical SVC on the test dataset:     0.92


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap
from qiskit_aer import AerSimulator


from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC


#Set random seed for reproducibility
np.random.seed(42)

#Create GPU-enabled AerSimulator backend
simulator = AerSimulator(
    method='statevector',
    device='GPU',
    # Optional: Enable cuStateVec for additional acceleration
    cuStateVec_enable=True,
    # Optional: Enable batched GPU execution for multiple shots
    batched_shots_gpu=True,
    # Optional: Set precision (single/double)
    precision='double'
)



sampler = Sampler()



num_features = 2
feature_map = ZFeatureMap(feature_dimension=num_features, reps=2)

quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
)



qsvc = QSVC(quantum_kernel=quantum_kernel)


print("Training QSVC on GPU...")
qsvc.fit(train_features, train_labels)


train_score = qsvc.score(train_features, train_labels)
test_score = qsvc.score(test_features, test_labels)

print(f"Training accuracy: {train_score:.2f}")
print(f"Testing accuracy: {test_score:.2f}")

In [ ]:

svc_feature_map = ZFeatureMap(feature_dimension= train_features.shape[1], reps=2)

sampler = Sampler()

svc_kernel = FidelityQuantumKernel(feature_map=svc_feature_map)
print(svc_feature_map.decompose())
print(svc_kernel.evaluate)

svc = SVC(kernel=svc_kernel.evaluate)
svc.fit(train_features, train_labels)
svc_score = svc.score(test_features, test_labels)
print(svc_score)

qsvc= QSVC(quantum_kernel = svc_kernel)
qsvc.fit(X=train_features, y=train_labels)
qsvc_score=qsvc_score.score(X=test_features, y=test_labels)
print(qsvc_score)

     ┌───┐┌─────────────┐┌───┐┌─────────────┐
q_0: ┤ H ├┤ P(2.0*x[0]) ├┤ H ├┤ P(2.0*x[0]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ H ├┤ P(2.0*x[1]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_2: ┤ H ├┤ P(2.0*x[2]) ├┤ H ├┤ P(2.0*x[2]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_3: ┤ H ├┤ P(2.0*x[3]) ├┤ H ├┤ P(2.0*x[3]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_4: ┤ H ├┤ P(2.0*x[4]) ├┤ H ├┤ P(2.0*x[4]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_5: ┤ H ├┤ P(2.0*x[5]) ├┤ H ├┤ P(2.0*x[5]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_6: ┤ H ├┤ P(2.0*x[6]) ├┤ H ├┤ P(2.0*x[6]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_7: ┤ H ├┤ P(2.0*x[7]) ├┤ H ├┤ P(2.0*x[7]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_8: ┤ H ├┤ P(2.0*x[8]) ├┤ H ├┤ P(2.0*x[8]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_9: ┤ H ├┤ P(2.0*x[9]) ├┤ H ├┤ P(2.0*x[9]) ├
     └───┘└─────────────┘└───┘└─────────────┘
<bound method FidelityQuantumKerne

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap
from qiskit_aer import AerSimulator


from qiskit_machine_learning.kernels import StatevectorKernel
from qiskit_machine_learning.algorithms import QSVC


#Set random seed for reproducibility
np.random.seed(42)

#Create GPU-enabled AerSimulator backend
simulator = AerSimulator(
    method='statevector',
    device='GPU',
    # Optional: Enable cuStateVec for additional acceleration
    cuStateVec_enable=True,
    # Optional: Enable batched GPU execution for multiple shots
    batched_shots_gpu=True,
    # Optional: Set precision (single/double)
    precision='double'
)



sampler = Sampler()



num_features = 2
feature_map = ZFeatureMap(feature_dimension=num_features, reps=2)

quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
)



qsvc = QSVC(quantum_kernel=quantum_kernel)


print("Training QSVC on GPU...")
qsvc.fit(train_features, train_labels)


train_score = qsvc.score(train_features, train_labels)
test_score = qsvc.score(test_features, test_labels)

print(f"Training accuracy: {train_score:.2f}")
print(f"Testing accuracy: {test_score:.2f}")

In [8]:
# Set up the quantum feature map
num_features = X.shape[1]  # Number of features from your dataset
feature_dimension = num_features

# Create Z Feature Map
z_feature_map = ZFeatureMap(
    feature_dimension=feature_dimension,
    reps=1,  # Number of repetitions of the feature map
    data_map_func=None,  # Use default data mapping
    parameter_prefix='x'
)

# Create ansatz (variational circuit)
ansatz = RealAmplitudes(
    num_qubits=feature_dimension,
    reps=1,  # Number of layers in the ansatz
    entanglement='linear'  # Type of entanglement
)

# Set up the quantum simulator
simulator = AerSimulator()
sampler = Sampler()

# Create the Variational Quantum Classifier
vqc = MLVQC(
    feature_map=z_feature_map,
    ansatz=ansatz,
    optimizer=COBYLA(maxiter=100),  # Classical optimizer
    sampler=sampler,
    callback=None  # Optional callback for monitoring training
)

# Train the quantum model
print("Training Quantum ML model...")
vqc.fit(train_features, train_labels)

# Make predictions
train_pred_quantum = vqc.predict(train_features)
test_pred_quantum = vqc.predict(test_features)

# Calculate scores
train_score_quantum = vqc.score(train_features, train_labels)
test_score_quantum = vqc.score(test_features, test_labels)

print(f"Quantum ML on training dataset: {train_score_quantum:.2f}")
print(f"Quantum ML on test dataset: {test_score_quantum:.2f}")

# Optional: Print circuit information
print(f"\nQuantum Circuit Information:")
print(f"Number of qubits: {z_feature_map.num_qubits}")
print(f"Feature map depth: {z_feature_map.depth()}")
print(f"Ansatz depth: {ansatz.depth()}")
print(f"Total parameters: {ansatz.num_parameters}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Training Quantum ML model...


KeyboardInterrupt: 